# Repeated Measures ANOVA for Movement
The purpose of this workbook is to use the seperate animal data from eahc anima and condition to find the repeated measures ANOVA values for the movement assays of average velocity and average hunch over time during the experiment.

## Data Input
This section of the workbook inports the data files, and rearranges them so they can more easily be worked with

In [1]:
# This cell will import libraries
import numpy as np
import pandas as pd
import scipy.integrate
from scipy import stats
from matplotlib import pyplot as plt
from os import listdir
from IPython.display import display
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

font = {'fontname':'Liberation Serif'}

In [2]:
# This cell loads the files generated in the Movement Analysis workbook

# File paths to be used while importing and merging
data_path_pd='/home/sam/Science/microdialysis/MSDissertation/Behavior/DLC_CSVs/Output/PD/ind/'
data_path_sham='/home/sam/Science/microdialysis/MSDissertation/Behavior/DLC_CSVs/Output/Sham/ind/'


# Code for finding names of all CSVs in the specified folder 
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

file_names_pd = find_csv_filenames(data_path_pd, suffix=".csv" )
file_names_sham=find_csv_filenames(data_path_sham, suffix=".csv" )
print("This directory contains",str((len(file_names_pd)+len(file_names_sham))),"files which have been loaded")


sham_COM=pd.DataFrame()
sham_Hunch=pd.DataFrame()
df=pd.DataFrame()



for name in file_names_sham:
    if name[0:3]=="COM":
        df=pd.read_csv(data_path_sham+name)
        for ii in range(len(df)):
            df.loc[ii, "Animal"]=name[-6:-4]

        sham_COM=sham_COM.append(df,ignore_index=True)

    if name[0:3]=="Hun":
        df=pd.read_csv(data_path_sham+name)
        for ii in range(len(df)):
            df.loc[ii, "Animal"]=name[-6:-4]

        sham_Hunch=sham_Hunch.append(df)


    
pd_COM=pd.DataFrame()
pd_Hunch=pd.DataFrame()


for name in file_names_pd:
    if name[0:3]=="COM":
        df=pd.read_csv(data_path_pd+name)
        for ii in range(len(df)):
            df.loc[ii, "Animal"]=name[-6:-4]

        pd_COM=pd_COM.append(df,ignore_index=True)

    if name[0:3]=="Hun":
        df=pd.read_csv(data_path_pd+name)
        for ii in range(len(df)):
            df.loc[ii, "Animal"]=name[-6:-4]
  
        pd_Hunch=pd_Hunch.append(df)

sham_COM=sham_COM.reset_index(drop=True)
sham_Hunch=sham_Hunch.reset_index(drop=True)
pd_COM=pd_COM.reset_index(drop=True)
pd_Hunch=pd_Hunch.reset_index(drop=True)

print("Files loaded")

This directory contains 8 files which have been loaded
Files loaded


In [3]:
#This cell will stack the Hunch and COM files such that the unstimulated period procedes the stimulated period
    #this allows them to be used to more easily compare Sham animals and Parkinsonian animals

def stacker(df,name,group):
    store=pd.DataFrame()
    for index in range(len(df)):
        store.loc[index,name]=df.loc[index,"Unstimulated"]
        store.loc[index,"Time"]=int(df.loc[index,"Time (min)"])
        store.loc[index,"Animal"]=df.loc[index,"Animal"]
        store.loc[index,"Condition"]="Unstimulated"
        store.loc[index,"Group"]=group

    for index in range(len(df)):
#         The stack_index is the continuation of hte end of the index range in the proceding part of the df
        stack_index=len(df)+index
        store.loc[stack_index,name]=df.loc[index,"Stimulated"]
        store.loc[stack_index,"Animal"]=df.loc[index,"Animal"]

#         Time is incremented by adding each time value to the highest possible so it continues in sequence
        store.loc[stack_index,"Time"]=int(df.loc[index,"Time (min)"])+int(df.loc[(len(df)-1),"Time (min)"])
        store.loc[stack_index,"Condition"]="Stimulated"
        store.loc[index,"Group"]=group
    return(store)


x=stacker(pd_Hunch,"Hunch", "PD")
y=stacker(sham_Hunch,"Hunch", "Sham")
Hunch=pd.concat([x,y], sort=True)
Hunch["Group"][Hunch["Group"]!="PD"]="Sham"

x=stacker(pd_COM,"COM","PD")
y=stacker(sham_COM,"COM","Sham")
COM=x.append(y, sort=True)
COM["Group"][COM["Group"]!="PD"]="Sham"


/home/sam/anaconda3/envs/jup/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sam/anaconda3/envs/jup/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# R
The below cells run stastics through R

In [4]:
%load_ext rpy2.ipython

In [ ]:
def binaryer(df1, name):

    df=df1.reset_index(drop=True)
    for ii in range(len(df)):
        df.loc[ii,name]=round(df.loc[ii,name],3)
        df.loc[ii,"Time"]=df.loc[ii,"Time"].astype(int)
#         print(type(df.loc[ii,"Time"]))
        
        if df.loc[ii,"Animal"] == "D3":
            df.loc[ii,"Animal"] = 0
        elif df.loc[ii,"Animal"] == "D2":
            df.loc[ii,"Animal"] = 1
        elif df.loc[ii,"Animal"] == "D4":
            df.loc[ii,"Animal"] = 2
        elif df.loc[ii,"Animal"] == "12":
            df.loc[ii,"Animal"] = 3
        elif df.loc[ii,"Animal"] == "11":
            df.loc[ii,"Animal"] = 4
            
        if df.loc[ii,"Condition"] == "Unstimulated":
            df.loc[ii,"Condition"] = 0
        else:
            df.loc[ii,"Condition"] = 1
        
        if df.loc[ii,"Group"] == "PD":
            df.loc[ii,"Group"] = 0
        else:
            df.loc[ii,"Group"] = 1
    return(df)

# def musher(df):
#     for animal in range(len(df["Animal"].unique())):
#         if df["Animal"]==
    
#     for ii in range(len(df["Time"].unique())):
#         if df.loc[ii,"Time"]== 1+ df.loc[ii-1,"Time"]:
            




hunch=binaryer(Hunch, "Hunch")


display(hunch.head)
com.to_csv('hunch_pspp.csv')



In [6]:
xxx=com["Animal"].unique()

com["Animal"][24]==xxx[0]

com.loc[0:22,"Time"].mean()

def musher(df,name):
    mush=pd.DataFrame()

    for animal in df["Animal"].unique():
        seg=df[df["Animal"]== animal]
        
        try:
            mush.loc[0,name]=seg.loc[(seg["Time"][0]):seg["Time"][9],name].mean()
            mush.loc[0,"Aniamal"]=animal
            mush.loc[0,"Time"]=10
            mush.loc[1,name]=seg.loc[seg["Time"][10]:seg["Time"][19],name].mean()
            mush.loc[1,"Aniamal"]=animal
            mush.loc[1,"Time"]=20
            mush.loc[2,name]=seg.loc[seg["Time"][20]:seg["Time"][22],name].mean()
            mush.loc[2,"Aniamal"]=animal
            mush.loc[2,"Time"]=30
            mush.loc[3,name]=seg.loc[seg["Time"][30]:seg["Time"][39],name].mean()
            mush.loc[3,"Aniamal"]=animal
            mush.loc[3,"Time"]=40
            mush.loc[4,name]=seg.loc[seg["Time"][40]:seg["Time"][49],name].mean()
            mush.loc[4,"Aniamal"]=animal
            mush.loc[4,"Time"]=50
            mush.loc[5,name]=seg.loc[seg["Time"][50]:seg["Time"][59],name].mean()
            mush.loc[5,"Aniamal"]=animal
            mush.loc[5,"Time"]=60
            print("end")
            
        except:
          print("An exception occurred"+animal)
#         mush.loc[0,"Time"]=seg.loc[0:9,"Time"].sum()
#         mush.loc[1,"Time"]=seg.loc[10:19,"Time"].sum()
#         mush.loc[2,"Time"]=seg.loc[20:29,"Time"].sum()
#         mush.loc[3,"Time"]=seg.loc[30:39,"Time"].sum()
#         mush.loc[4,"Time"]=seg.loc[40:49,"Time"].sum()
#         mush.loc[5,"Time"]=seg.loc[50:-1,"Time"].sum()
        display(mush)
musher(COM,"COM")
display(Hunch["Time"][9])

NameError: name 'com' is not defined

In [7]:
%%R -i Hunch -i COM
#This R cell calculates the repeated measures ANOVA value for the COM

print("COM Repeated Measures ANOVA values:")
summary(aov(COM ~ interaction(Time) * interaction(Group) , data=COM))

[1] "COM Repeated Measures ANOVA values:"
                                      Df Sum Sq Mean Sq F value  Pr(>F)   
interaction(Time)                     58  129.9   2.239   0.926 0.62373   
interaction(Group)                     1   21.9  21.906   9.054 0.00314 **
interaction(Time):interaction(Group)  29   10.9   0.377   0.156 1.00000   
Residuals                            133  321.8   2.419                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [8]:
%%R -i Hunch -i COM
#This R cell calculates the repeated measures ANOVA value for the Hunchin

print("Hunch Repeated Measures ANOVA values:")
summary(aov(Hunch ~ interaction(Time) * interaction(Group) * interaction(Condition), data=Hunch))

[1] "Hunch Repeated Measures ANOVA values:"
                                      Df Sum Sq Mean Sq F value   Pr(>F)    
interaction(Time)                     58  919.4    15.9   1.359   0.0769 .  
interaction(Group)                     1  753.4   753.4  64.571 4.59e-13 ***
interaction(Condition)                 1    0.0     0.0   0.000   0.9843    
interaction(Time):interaction(Group)  29   86.2     3.0   0.255   1.0000    
Residuals                            132 1540.1    11.7                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [9]:
com_rounded=[1,2,1,3,3,3,1,1,2,2,2,6,0,0,0,0,1,1,1,0,1,0,0,0]



In [11]:
OX=pd.read_csv("/home/sam/Documents/IIN-ELS/Defesa/6-OHDA_ox.csv")

OX0 = OX[OX["Time"]==0] 

OX0_clean= OX0[OX0["Note"]!="UV"]
OX0_clean= OX0[OX0["Note"]!="FRESH"]
OX0_clean=OX0_clean.drop(columns="Note")
OX0_clean.columns=["sample","Day","Time","A"]

OX0_clean=OX0_clean.reset_index(drop=True)

OX0_clean.head()

# display(stats.ttest_rel()

,sample,Day,Time,A
0,A,0,0,5.30
1,B,0,0,5.52
2,C,0,0,4.84
3,A,1,0,5.26
4,B,1,0,4.93


In [12]:
%%R -i OX0_clean
# -i Hunch -i com

#This cell is running R code, after loading R from the cell above
ox <- OX0_clean



print(class(ox['Day']))


pairwise.t.test(ox$A, ox$Day, p.adj="holm")
#LSD.test(ox$A,ox$Day)

ox$Day<-factor(ox$Day)
summary(aov(A ~ Day, data=ox))

[1] "data.frame"
            Df Sum Sq Mean Sq F value Pr(>F)
Day          9  29.26   3.251   1.691  0.149
Residuals   23  44.23   1.923               
